In [1]:
import torch 
import pandas as pd
import torch.nn as nn
import time
import io
import joblib
import argparse

from torchsummary import summary
from tqdm import tqdm
from tabulate import tabulate
from sklearn.preprocessing import *

from AutoCleanse.utils import *
from AutoCleanse.dataloader import PlainDataset, DataLoader
from AutoCleanse.autoencoder import *
from AutoCleanse.loss_model import loss_CEMSE
from AutoCleanse.preprocessor import Preprocessor
from AutoCleanse.anonymize import anonymize
from AutoCleanse.bucketfs_client import *

Device configuration

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)

## Setup directory path

In [3]:
import os
PROJECT_DIR = os.getcwd()
os.chdir(PROJECT_DIR)
DATASET_DIR = os.path.join(PROJECT_DIR,'dataset')
EVAL_DIR = os.path.join(PROJECT_DIR,'evaluate')

## Preparing data

Load dataframe and group features by their type

In [4]:
df = pd.read_csv(os.path.join(DATASET_DIR,'adult.csv')).drop(columns=['fnlwgt','income'])
continous_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_columns = df.select_dtypes(include=['object', 'bool']).columns.tolist()
og_columns = df.columns.to_list()
df = df[continous_columns+categorical_columns]

Data preprocessing

In [16]:
scaler = MinMaxScaler()
onehotencoder = OneHotEncoder(handle_unknown='ignore',sparse_output=False)
preprocessor = Preprocessor(scaler,onehotencoder)

X_train,X_val,X_test = preprocessor.split(df=df,
                                        train_ratio=0.7,
                                        val_ratio=0.15,
                                        test_ratio=0.15,
                                        random_seed=42)
X_dirty = corrupt(X_test,0.2,42)
X_dirty
# X_train = preprocessor.fit_transform(input_df=X_train,
#                                     continous_columns=continous_columns,
#                                     categorical_columns=categorical_columns)

# X_val = preprocessor.transform(input_df=X_val,    
#                                continous_columns=continous_columns,
#                                categorical_columns=categorical_columns)                          

# X_test = preprocessor.transform(input_df=X_test,   
#                                 continous_columns=continous_columns,
#                                 categorical_columns=categorical_columns)  

# X_dirty = preprocessor.transform(input_df=X_dirty,   
#                                 continous_columns=continous_columns,
#                                 categorical_columns=categorical_columns)

# categories = preprocessor.encoder.categories_

,age,education.num,capital.gain,capital.loss,hours.per.week,workclass,education,marital.status,occupation,relationship,race,sex,native.country
28296,48,5,0,-7002,20,?,9th,Separated,?,Not-in-family,Amer-Indian-Eskimo,Female,United-States
28217,28,-7002,0,0,40,?,HS-grad,Separated,?,NaN,White,Female,United-States
8054,38,5,0,0,40,Self-emp-not-inc,9th,Divorced,Craft-repair,Not-in-family,White,Male,United-States
4223,77,9,-7002,0,20,NaN,HS-grad,Never-married,Machine-op-inspct,Not-in-family,White,Male,United-States
22723,33,11,0,0,84,Private,Assoc-voc,Married-civ-spouse,NaN,Husband,White,Male,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21656,25,10,0,0,35,Self-emp-inc,Some-college,Never-married,Sales,Own-child,White,Female,NaN
28318,19,7,0,0,-7002,Private,11th,Never-married,NaN,NaN,White,Male,United-States
32547,39,-7002,-7002,0,20,Local-gov,Assoc-acdm,Married-civ-spouse,Adm-clerical,Wife,NaN,Female,United-States
9375,42,9,0,0,40,Private,HS-grad,NaN,NaN,Husband,White,NaN,NaN


(Optional) Check save/load function of preprocessor

Both functiona take in 2 parameters:

    - Suffix of the preprocessor name, in the example below would be **preprocessor_main.pkl**
    - Save/load location: can either be "local" to save/load in the home folder or "bucketfs" to save/load to/from Exasol BucketFS

In [ ]:
preprocessor.save("main","local")
preprocessor.save("main","bucketfs")
preprocessor = Preprocessor(scaler=MinMaxScaler(),encoder=OneHotEncoder(sparse=False))
preprocessor2.load("main","local")
preprocessor2.load("main","bucketfs")

Convert dataframes into datasets, and create dataloaders

In [ ]:
batch_size = 64

In [ ]:
train_dataset = PlainDataset(X_train)
val_dataset = PlainDataset(X_val)
test_dataset = PlainDataset(X_test)
dirty_dataset = PlainDataset(X_dirty)

def custom_collate_fn(batch):
    tensor_data = torch.stack([item[0] for item in batch])
    indices = [item[1] for item in batch]
    return tensor_data, indices

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True,collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True, collate_fn=custom_collate_fn)
dirty_loader = DataLoader(dirty_dataset, batch_size=batch_size, shuffle=False, drop_last=True, collate_fn=custom_collate_fn)

## Instantiate model

In [ ]:
layers = [X_train.shape[1],1024,128]   
wlc = (1,1) 

autoencoder = Autoencoder(layers=layers,dropout_enc=[(0,0.0)],dropout_dec=[(0,0.1)], batch_norm=True, \
                          learning_rate=1e-4,weight_decay=1e-5,l1_strength=1e-5,l2_strength=1e-5)

In [ ]:
summary(autoencoder.to(device),torch.tensor(X_train.values).float().to(device).shape[1:])

(Optional) Model can be loaded from checkpoint after instantiation

The function takes in 2 parameters:

    - Suffix of the preprocessor name, in the example below would be **autoencoder_main.pkl**
    - Save/load location: can either be "local" to load in the home folder or "bucketfs" to load from Exasol BucketFS

In [ ]:
autoencoder.load("local","main")

### Train the model

In [ ]:
autoencoder.train_model(
      patience=10,
      num_epochs=100,
      batch_size=batch_size,
      train_loader=train_loader,
      val_loader=val_loader,
      continous_columns=continous_columns, 
      categorical_columns=categorical_columns, 
      categories=categories,
      device=device,
      wlc=wlc)

(Optional) Model can be saved after training

The function takes in 2 parameters:

    - Suffix of the preprocessor name, in the example below would be **autoencoder_main.pkl**
    - Save/load location: can either be "local" to load in the home folder or "bucketfs" to load from Exasol BucketFS

In [ ]:
autoencoder.save("local","main")

### Use trained model to clean data

In [ ]:
cleaned_data = autoencoder.clean(dirty_loader=dirty_loader,
                                test_loader=test_loader,
                                df=X_dirty,
                                batch_size=batch_size,
                                continous_columns=continous_columns, 
                                categorical_columns=categorical_columns, 
                                og_columns=og_columns,
                                onehotencoder=preprocessor.encoder, 
                                scaler=preprocessor.scaler,
                                device=device) 

In [ ]:
# original data
print(tabulate(df.loc[[28296,28217,8054,4223,22723],og_columns],headers=og_columns,tablefmt="simple",maxcolwidths=[None, 4]))

In [ ]:
# cleaned data
print(tabulate(cleaned_data.loc[[28296,28217,8054,4223,22723]],headers=cleaned_data.columns.to_list(),tablefmt="simple",maxcolwidths=[None, 4]))

### Use trained model to anonymize data

In [ ]:
anonymized_data = autoencoder.anonymize(df=X_test,
                                        data_loader=test_loader,
                                        batch_size=batch_size,
                                        device=device)

In [ ]:
# anonymized data
print(tabulate(anonymized_data.round(decimals=4).iloc[:5,:32],headers=anonymized_data.columns.to_list(),tablefmt="simple",maxcolwidths=[None, 6]))